In [2]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score
import fastapi
from typing import Optional

In [3]:
_x = [1]
_y = [1]
_prediction = ["True"]
_result = ["True"]
model = DummyClassifier(strategy="constant", constant="True")
model.fit(np.array([_x,_y]).reshape(1, -1), _result)


DummyClassifier(constant='True', strategy='constant')

In [4]:
def model_evaluation(y_test, y_pred) :
    print('Trained Model Test Data Accuracy Score :',accuracy_score(y_test, y_pred)*100)
    testacc=accuracy_score(y_test, y_pred)
    testrecall=recall_score(y_test, y_pred, pos_label="True")
    testprecision=precision_score(y_test, y_pred, pos_label="True")
    print(' ')
    print(classification_report(y_test, y_pred))
    return testacc, testrecall, testprecision

In [5]:
def create_eval_df(x, y, result) :
    data = {"x": x, "y": y, "result": result}
    df = pd.DataFrame(data)
    eval_df = df.copy()
    eval_df["predict"] = model.predict(data[x,y])
    eval_df["correct"] = eval_df["predict"] == eval_df["result"]
    eval_df["correctandtrue"] = eval_df["predict"].isin([True]) & eval_df["result"].isin([True])
    return eval_df

In [6]:
app = fastapi.FastAPI()


@app.get("/predict/")
def predict(id:int = 0 , x:float =0, y:float=0):
    print(id, x, y)
    _x.insert(id, x)
    _y.insert(id, y)
    prediction = model.predict(np.array([x,y]).reshape(1, -1))
    _prediction.insert(id, prediction[0])
    print(prediction[0])
    return dict([("prediction", prediction[0])])

@app.get("/result/")
def result(id:int, result:str):
    print(result)
    _result.insert(id, result)
    return dict([("message", "Copied result for {}".format(id))])

@app.get("/retrain/")
def retrain(last_n:int):
    available_data = min(len(_x), last_n)
    print ("RETRAINING over last {}".format(last_n))
    #print(_x)
    #print(_y)
    X = np.column_stack((_x, _y))[-available_data:]  #np.rec.fromarrays([_x, _y])[:(-1*last_n)]
    Y = np.array(_result)[-available_data:]  #np.rec.fromarrays([_result])[:(-1*last_n)]


    #print(X, Y)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.40)

    print(X_train)
    print(y_train)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    testacc, testrecall, testprecision = model_evaluation(y_test, y_pred)

    return {"message": "Model retrained", "accuracy": testacc, "recall": testrecall, "precision": testprecision}

@app.get("/new_model/")
def new_model(model:str, last_n:int, strategy:Optional[str] = None, k:Optional[int] = 0):
    print(model, last_n, strategy, k)
    if model == "Dummy" :
        model = DummyClassifier(strategy=strategy)
    elif model == "KNN" :
        model = KNeighborsClassifier(k=k)
    #model.set_params(model_params)

    message = retrain(last_n=last_n)

    returns = {"message": "Model trained {}".format(model), "train": message}
    print(returns)

    return returns

#@app.get("/evaluate_model/")
#def evaluate_model(last_n:int):
#    return {"message": "Item created", "last_n": last_n}

In [7]:
_x = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0]
_y = [1.0, 1.0, 2.0, 2.0, 3.0, 3.0]
_prediction = ["True", "False", "True", "False", "True", "False"]
_result = ["True", "False", "True", "False", "True", "False"]


In [8]:
#print(predict(1, 1, 1))
#result(1, True)
new_model("Dummy", 50, "stratified")

Dummy 50 stratified 0
RETRAINING over last 50
[[5. 3.]
 [3. 2.]
 [2. 1.]]
['True' 'True' 'False']
Trained Model Test Data Accuracy Score : 33.33333333333333
 
              precision    recall  f1-score   support

       False       0.00      0.00      0.00         2
        True       0.33      1.00      0.50         1

    accuracy                           0.33         3
   macro avg       0.17      0.50      0.25         3
weighted avg       0.11      0.33      0.17         3

{'message': "Model trained DummyClassifier(strategy='stratified')", 'train': {'message': 'Model retrained', 'accuracy': 0.3333333333333333, 'recall': 1.0, 'precision': 0.3333333333333333}}


C:\Users\jagru\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\jagru\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\jagru\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\metrics\_classificati

{'message': "Model trained DummyClassifier(strategy='stratified')",
 'train': {'message': 'Model retrained',
  'accuracy': 0.3333333333333333,
  'recall': 1.0,
  'precision': 0.3333333333333333}}

In [9]:
@app.get("/items/")
async def read_item(skip: int = 0, limit: int = 10):
    return {"skip": skip, "limit": limit}

In [10]:
import asyncio
import uvicorn

if __name__ == "__main__":
    config = uvicorn.Config(app, host="127.0.0.1", port=8001)
    server = uvicorn.Server(config)
    loop = asyncio.get_running_loop()
    loop.create_task(server.serve())

INFO:     Uvicorn running on http://127.0.0.1:8001 (Press CTRL+C to quit)


In [12]:
!curl "http://127.0.0.1:8000/retrain/?model_type=KNN&last_n=10&k=5"

{"message":"Model retrained","accuracy":0.5238095238095238,"recall":1.0,"precision":0.5238095238095238}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   103  100   103    0     0   4018      0 --:--:-- --:--:-- --:--:--  4120


In [13]:
!curl "http://127.0.0.1:8000/retrain/?model_type=Dummy&last_n=8&strategy=stratified"

{"message":"Model retrained","accuracy":0.3333333333333333,"recall":1.0,"precision":0.3333333333333333}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   103  100   103    0     0   4991      0 --:--:-- --:--:-- --:--:--  5150
